In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Checkpoint Evaluación de modelos e importancia de features

## Dataset

**Introducción**

La Organización Mundial de la Salud ha estimado que 12 millones de muertes ocurren en todo el mundo, cada año debido a enfermedades del corazón. La mitad de las muertes en los Estados Unidos y otros países desarrollados se deben a enfermedades cardiovasculares. El pronóstico temprano de las enfermedades cardiovasculares puede ayudar a tomar decisiones sobre los cambios en el estilo de vida en pacientes de alto riesgo y, a su vez, reducir las complicaciones. Esta investigación tiene la intención de identificar los factores más relevantes / de riesgo de enfermedad cardíaca, así como predecir el riesgo general mediante distintos modelos de clasificación

**Fuente**

El conjunto de datos está disponible públicamente en el sitio web de Kaggle (https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset), y proviene de un estudio cardiovascular en curso en residentes de la ciudad de Framingham, Massachusetts. El objetivo de la clasificación es predecir si el paciente tiene riesgo de enfermedad coronaria (CHD) en los próximos 10 años. El conjunto de datos proporciona la información del paciente. Incluye más de 4000 registros y 15 atributos.

**Variables**

Cada atributo es un factor de riesgo potencial. Hay factores de riesgo demográficos, conductuales y médicos.

Demográficos:

• Sex: male or female(Nominal)

• Age: Age of the patient;(Continuous - Although the recorded ages have been truncated to whole numbers, the concept of age is continuous)
Behavioral

• Current Smoker: whether or not the patient is a current smoker (Nominal)

• Cigs Per Day: the number of cigarettes that the person smoked on average in one day.(can be considered continuous as one can have any number of cigarettes, even half a cigarette.)

Médicos (histórico):

• BP Meds: whether or not the patient was on blood pressure medication (Nominal)

• Prevalent Stroke: whether or not the patient had previously had a stroke (Nominal)

• Prevalent Hyp: whether or not the patient was hypertensive (Nominal)

• Diabetes: whether or not the patient had diabetes (Nominal)

Médicos (actual):

• Tot Chol: total cholesterol level (Continuous)

• Sys BP: systolic blood pressure (Continuous)

• Dia BP: diastolic blood pressure (Continuous)

• BMI: Body Mass Index (Continuous)

• Heart Rate: heart rate (Continuous - In medical research, variables such as heart rate though in fact discrete, yet are considered continuous because of large number of possible values.)

• Glucose: glucose level (Continuous)

Variable a predecir (target):

• 10 year risk of coronary heart disease CHD (binary: “1”, means “Yes”, “0” means “No”)



## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report


## Ejercicio 1 - Preparación de datos

1.1) Leamos los datos del archivo datasets_222487_478477_framingham.csv

1.2) ¿Qué porcentaje de registros hay en cada una de las categorías target?

1.3) ¿El dataset tiene datos faltantes?

1.4) Usemos `dropna` para eliminar los registros con valores faltantes, y volvamos a calcular el porcentaje de registros hay en cada una de las categorías target 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html


In [ ]:
data_raw = pd.read_csv('../Data/datasets_222487_478477_framingham.csv')
data_raw.shape

In [ ]:
data_raw.columns

Veamos qué procentaje de registros hay en cada una de las categorías target

In [ ]:
data_raw.TenYearCHD.value_counts() / data_raw.shape[0]

In [ ]:
data_raw.info()

In [ ]:
data = data_raw.dropna()
data.shape

In [ ]:
data.TenYearCHD.value_counts() / data.shape[0]

Vemos que en este caso, eliminando los registros que tienen algun valor nulo no cambiamos la proporción de registros en cada una de las categorías target

In [ ]:
data.dtypes

In [ ]:
data.head()

## Ejercicio 2 - Train Test Split + StandardScaler

Construir los conjuntos de entranamiento y test y usando StandardScaler normalizar las features

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
X=data.drop(["TenYearCHD"],axis=1)
y=data["TenYearCHD"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=717)
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

## Ejercicio 3 - Opcional (pero recomendado)

Escribir una función que reciba como parámetros 

* la instancia de un modelo
* X_train, los registros con los valores de las features predictoras en el dataset de entrenamiento;
* y_train, los registros con el valor del target en el dataset de entrenamiento;
* X_test, los registros con los valores de las features predictoras en el dataset de testing;
* y_test, los registros con el valor del target en el dataset de testing;
* gridSearch_params, los parámetros para usar en grid search para la instancia del modelo
* gridSearch_bagging_params, los parámetros para usar en grid search para bagging

La función debe entrenar 

* El modelo que recibe como parámetro usando X_train, y_train

* Grid Search Cross Validation KFold: `cv_KFold = KFold(n_splits=3, shuffle=True, random_state=371)`
    

* Grid Search CV Stratified KFold: `StratifiedKFold(n_splits=3, shuffle=True, random_state=371)`

* Bagging

* Bagging Grid Search Stratified Cross Validation: usando como base el mejor estimador devuelto en Grid Search CV Stratified KFold y `StratifiedKFold(n_splits=3, shuffle=True, random_state=371)`

Esta función debe devolver alguna estructura de datos, por ejemplo un diccionario, donde después podamos consultar el score en test de cada uno de los modelos, y los valores de media y desvío del score en los entrenamientos con grid search.


In [ ]:
def evaluate_model(model_instance, X_train, y_train, X_test, y_test, gridSearch_params, gridSearch_bagging_params):
    
    # entreno el modelo default
    model_instance.fit(X_train, y_train)
    
    # calculo el score sobre los datos de test
    score_default_test = model_instance.score(X_test, y_test)
    
    # calculo la matriz de confusión
    predictions_default = model_instance.predict(X_test)
    confusion_matrix_default = metrics.confusion_matrix(y_test, predictions_default)
    
    ###################################################
    
    # gridSearch KFold:    
    cv_KFold = KFold(n_splits=3, shuffle=True, random_state=371)
    grid_search_CV_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_KFold)    
    grid_search_CV_KFold_model.fit(X_train, y_train)        
    scores_KFold = cross_val_score(model_instance, X_train, y_train, cv=cv_KFold, n_jobs=-1)
    mean_score_grid_search_CV_KFold_model = scores_KFold.mean()
    std_score_grid_search_CV_KFold_model = scores_KFold.std()
        
    score_grid_search_CV_KFold_model = grid_search_CV_KFold_model.best_score_
    params_grid_search_CV_KFold_model = grid_search_CV_KFold_model.best_params_
    
    score_grid_search_CV_KFold_model_test = grid_search_CV_KFold_model.score(X_test, y_test)
    predictions_grid_search_CV_KFold_model = grid_search_CV_KFold_model.predict(X_test)    
    confusion_matrix_grid_search_CV_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_KFold_model)

    ###################################################
    
    # gridSearch Stratified KFold:    
    cv_Stratified_KFold =StratifiedKFold(n_splits=3, shuffle=True, random_state=371)
    grid_search_CV_Stratified_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)    
    grid_search_CV_Stratified_KFold_model.fit(X_train, y_train)        
    scores_Stratified_KFold = cross_val_score(model_instance, X_train, y_train, cv=cv_Stratified_KFold, n_jobs=-1)
    mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
    std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    
    
    score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_score_
    params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_params_
    
    score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model.score(X_test, y_test)
    predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.predict(X_test)
    confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

    ###################################################
    
    if gridSearch_bagging_params:

        # bagging

        bagging_model_default = BaggingClassifier(base_estimator = model_instance)
        bagging_model_default.fit(X_train, y_train)
        score_bagging_model_default_test =  bagging_model_default.score(X_test, y_test)
        
        predictions_bagging_model_default = bagging_model_default.predict(X_test)
        confusion_matrix_bagging_model_default = metrics.confusion_matrix(y_test, predictions_bagging_model_default)    

        ###################################################

        # bagging Stratified KFold cross validation usando de base el mejor modelo de gridsearch estratificado
        base_estimator_stratified_grid_search = grid_search_CV_Stratified_KFold_model.best_estimator_
        cv_Stratified_KFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=371)
        grid_search_bagging_model = GridSearchCV(BaggingClassifier(base_estimator = base_estimator_stratified_grid_search),
                               gridSearch_bagging_params, n_jobs=-1, cv = cv_Stratified_KFold)

        grid_search_bagging_model.fit(X_train, y_train)
        
        score_grid_search_bagging_model_test = grid_search_bagging_model.score(X_test, y_test)
        predictions_grid_search_bagging_model = grid_search_bagging_model.predict(X_test)
        confusion_matrix_grid_search_bagging_model = metrics.confusion_matrix(y_test, predictions_grid_search_bagging_model)
        scores_bagging_Stratified_KFold = cross_val_score(BaggingClassifier(base_estimator = base_estimator_stratified_grid_search),
                                                          X_train, y_train, cv=cv_Stratified_KFold, n_jobs=-1)
        mean_score_bagging_grid_search_CV_Stratified_KFold_model = scores_bagging_Stratified_KFold.mean()
        std_score_bagging_grid_search_CV_Stratified_KFold_model = scores_bagging_Stratified_KFold.std()    
        
        best_score_bagging_grid_search_CV_Stratified_KFold_model = grid_search_bagging_model.best_score_
        best_params_bagging_grid_search_CV_Stratified_KFold_model = grid_search_bagging_model.best_params_
    
    else:
        
        score_bagging_model_default_test = None
        confusion_matrix_bagging_model_default = None
        mean_score_bagging_grid_search_CV_Stratified_KFold_model = None
        std_score_bagging_grid_search_CV_Stratified_KFold_model = None
        confusion_matrix_grid_search_bagging_model = None
        best_score_bagging_grid_search_CV_Stratified_KFold_model = None
        score_grid_search_bagging_model_test = None
        best_params_bagging_grid_search_CV_Stratified_KFold_model = None

                                                                                                                                     
    ###################################################
    
    
    # armo un diccionario con todos los valores de performance que calculé para los modelos
    result = {
        'default': {
            'score': score_default_test,
            'confusion_matrix': confusion_matrix_default            
        },
        'cv_kfold': {
            'mean_score_grid_search': mean_score_grid_search_CV_KFold_model,
            'std_score_grid_search': std_score_grid_search_CV_KFold_model,
            'best_score_grid_search': score_grid_search_CV_KFold_model,
            'score': score_grid_search_CV_KFold_model_test,
            'confusion_matrix': confusion_matrix_grid_search_CV_KFold_model           
        },
        'cv_stratified_kfold': {
            'mean_score_grid_search': mean_score_grid_search_CV_Stratified_KFold_model,
            'std_score_grid_search': std_score_grid_search_CV_Stratified_KFold_model,
            'best_score_grid_search': score_grid_search_CV_Stratified_KFold_model,
            'score': score_grid_search_CV_Stratified_KFold_model_test,
            'confusion_matrix': confusion_matrix_grid_search_CV_Stratified_KFold_model           
        },
        'bagging': {
            'score': score_bagging_model_default_test,
            'confusion_matrix': confusion_matrix_bagging_model_default           
        },
        'bagging_cv_stratified_kfold': {
            'mean_score_grid_search': mean_score_bagging_grid_search_CV_Stratified_KFold_model,
            'std_score_grid_search': std_score_bagging_grid_search_CV_Stratified_KFold_model,
            'best_score_grid_search': best_params_bagging_grid_search_CV_Stratified_KFold_model,
            'score': score_grid_search_bagging_model_test,
            'confusion_matrix': confusion_matrix_grid_search_bagging_model           
        }
        
    }
    
    return result
        
#           'trained_model_instance': model_instance, 
#              'score_default': score_default_test,
#              'confusion_matrix_default': confusion_matrix_default,
#              'mean_score_grid_search_CV_KFold_model': mean_score_grid_search_CV_KFold_model, 
#              'std_score_grid_search_CV_KFold_model': std_score_grid_search_CV_KFold_model, 
#              'best_score_grid_search_CV_KFold_model': score_grid_search_CV_KFold_model,
#              'best_params_grid_search_CV_KFold_model': params_grid_search_CV_KFold_model,              
#              'score_grid_search_CV_KFold_model_test': score_grid_search_CV_KFold_model_test,
#              'confusion_matrix_grid_search_CV_KFold_model': confusion_matrix_grid_search_CV_KFold_model,
        
#              'mean_score_grid_search_CV_Stratified_KFold_model': mean_score_grid_search_CV_Stratified_KFold_model, 
#              'std_score_grid_search_CV_Stratified_KFold_model': std_score_grid_search_CV_Stratified_KFold_model, 
#              'best_score_grid_search_CV_Stratified_KFold_model': score_grid_search_CV_Stratified_KFold_model,
#              'best_params_grid_search_CV_Stratified_KFold_model': params_grid_search_CV_Stratified_KFold_model,              
#              'score_grid_search_CV_Stratified_KFold_model_test': score_grid_search_CV_Stratified_KFold_model_test,
#              'confusion_matrix_grid_search_CV_Stratified_KFold_model': confusion_matrix_grid_search_CV_Stratified_KFold_model,
        
#              'score_bagging_default': score_bagging_model_default_test,
#              'confusion_matrix_bagging_default': confusion_matrix_bagging_model_default,
        
#              'mean_score_bagging_grid_search_CV_Stratified_KFold_model': mean_score_bagging_grid_search_CV_Stratified_KFold_model, 
#              'std_score_bagging_grid_search_CV__StratifiedKFold_model': std_score_bagging_grid_search_CV_Stratified_KFold_model, 
#              'best_score_bagging_grid_search_CV_Stratified_KFold_model': best_score_bagging_grid_search_CV_Stratified_KFold_model,
#              'best_params_bagging_grid_search_CV_Stratified_KFold_model': best_params_bagging_grid_search_CV_Stratified_KFold_model,              
#              'score_bagging_grid_search_CV_Stratified_KFold_model_test': score_grid_search_bagging_model_test,
#              'confusion_matrix_bagging_grid_search_CV_Stratified_KFold_model': confusion_matrix_grid_search_bagging_model,

    
    
    
    
    

## Ejercicio 4

Entrenar un modelo de clasificación basado en árboles, usando bagging y grid search cross validation estratificado sobre los parámetros de bagging, y calcular el score en test de cada uno de los modelos, y los valores de media y desvío del score en los entrenamientos con grid search cross validation.

Los parámetros de GridSearchCV que vamos a probar son 

``params =  {'criterion': ['gini', 'entropy'],
          'splitter': ['best', 'random'],
          'max_depth': [None, 5, 10],
          'min_samples_split': [2, 5],
          'min_samples_leaf': [1, 2, 3]}``

Los parámetros de bagging que vamos a probar son

<code>
bagging_params = {'n_estimators': [10, 100],

                  'max_samples': [0.01, 1.0],
                  
                  'max_features': [0.3, 1.0],
                  
                  'bootstrap_features': [True, False]}
</code>

Si resolvieron el ejercicio 3, pueden usar esa función. 

Si no, pueden hacer cada uno de los entrenamientos en la misma forma que venimos haciendo en las prácticas anteriores

In [ ]:
model_instance = DecisionTreeClassifier()
x_train = X_train_sc
x_test =  X_test_sc
gridSearch_params = {'criterion': ['gini', 'entropy'],
          'splitter': ['best', 'random'],
          'max_depth': [None, 5, 10],
          'min_samples_split': [2, 5],
          'min_samples_leaf': [1, 2, 3]}

gridSearch_bagging_params = {'n_estimators': [10, 100],
                  'max_samples': [0.01, 1.0],                  
                  'max_features': [0.3, 1.0],                  
                  'bootstrap_features': [True, False]}


In [ ]:
#usando la función:

decision_tree_metrics = evaluate_model(model_instance, x_train, y_train, x_test, y_test, gridSearch_params, gridSearch_bagging_params)

decision_tree_metrics

In [ ]:
# en la forma clásica

model_instance.fit(x_train, y_train)

# calculo el score sobre los datos de test
score_default_test = model_instance.score(x_test, y_test)

# calculo la matriz de confusión
predictions_default = model_instance.predict(X_test)
confusion_matrix_default = metrics.confusion_matrix(y_test, predictions_default)
    
###################################################
    
# gridSearch KFold:    
cv_KFold = KFold(n_splits=3, shuffle=True, random_state=371)
grid_search_CV_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_KFold)    
grid_search_CV_KFold_model.fit(x_train, y_train)        
scores_KFold = cross_val_score(model_instance, x_train, y_train, cv=cv_KFold, n_jobs=-1)
mean_score_grid_search_CV_KFold_model = scores_KFold.mean()
std_score_grid_search_CV_KFold_model = scores_KFold.std()

score_grid_search_CV_KFold_model = grid_search_CV_KFold_model.best_score_
params_grid_search_CV_KFold_model = grid_search_CV_KFold_model.best_params_

score_grid_search_CV_KFold_model_test = grid_search_CV_KFold_model.score(X_test, y_test)
predictions_grid_search_CV_KFold_model = grid_search_CV_KFold_model.predict(X_test)    
confusion_matrix_grid_search_CV_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_KFold_model)

###################################################
    
# gridSearch Stratified KFold:    
cv_Stratified_KFold =StratifiedKFold(n_splits=3, shuffle=True, random_state=371)
grid_search_CV_Stratified_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)    
grid_search_CV_Stratified_KFold_model.fit(x_train, y_train)        
scores_Stratified_KFold = cross_val_score(model_instance, x_train, y_train, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.mean()
std_score_grid_search_CV_Stratified_KFold_model = scores_Stratified_KFold.std()    

score_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_score_
params_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.best_params_

score_grid_search_CV_Stratified_KFold_model_test = grid_search_CV_Stratified_KFold_model.score(X_test, y_test)
predictions_grid_search_CV_Stratified_KFold_model = grid_search_CV_Stratified_KFold_model.predict(X_test)
confusion_matrix_grid_search_CV_Stratified_KFold_model = metrics.confusion_matrix(y_test, predictions_grid_search_CV_Stratified_KFold_model)

###################################################

# bagging

bagging_model_default = BaggingClassifier(base_estimator = model_instance)
bagging_model_default.fit(x_train, y_train)
score_bagging_model_default_test =  bagging_model_default.score(x_test, y_test)

predictions_bagging_model_default = bagging_model_default.predict(x_test)
confusion_matrix_bagging_model_default = metrics.confusion_matrix(y_test, predictions_bagging_model_default)    

###################################################

# bagging Stratified KFold cross validation usando de base el mejor modelo de gridsearch estratificado
base_estimator_stratified_grid_search = grid_search_CV_Stratified_KFold_model.best_estimator_
cv_Stratified_KFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=371)
grid_search_bagging_model = GridSearchCV(BaggingClassifier(base_estimator = base_estimator_stratified_grid_search),
                       gridSearch_bagging_params, n_jobs=-1, cv = cv_Stratified_KFold)

grid_search_bagging_model.fit(x_train, y_train)

score_grid_search_bagging_model_test = grid_search_bagging_model.score(x_test, y_test)
predictions_grid_search_bagging_model = grid_search_bagging_model.predict(x_test)
confusion_matrix_grid_search_bagging_model = metrics.confusion_matrix(y_test, predictions_grid_search_bagging_model)
scores_bagging_Stratified_KFold = cross_val_score(BaggingClassifier(base_estimator = base_estimator_stratified_grid_search),
                                                  x_train, y_train, cv=cv_Stratified_KFold, n_jobs=-1)
mean_score_bagging_grid_search_CV_Stratified_KFold_model = scores_bagging_Stratified_KFold.mean()
std_score_bagging_grid_search_CV_Stratified_KFold_model = scores_bagging_Stratified_KFold.std()    

best_score_bagging_grid_search_CV_Stratified_KFold_model = grid_search_bagging_model.best_score_
best_params_bagging_grid_search_CV_Stratified_KFold_model = grid_search_bagging_model.best_params_

## Ejercicio 5 _ Performance

En este ejercicio vamos a repasar dos métodos para evaluar la performance de un modelo.

* accuracy_score - Accuracy = (TP + TN)/(TP + TN + FP + FN)

* classification_report
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

Vamos a evaluarlas sobre el modelo de bagging Stratified KFold cross validation que usa de base el mejor modelo de decision tree resultado gridsearch estratificado `StratifiedKFold(n_splits=3, shuffle=True, random_state=371)`


In [ ]:
# el mejor modelo de gridsearch estratificado:

model_instance = DecisionTreeClassifier()
cv_Stratified_KFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=371)
grid_search_CV_Stratified_KFold_model = GridSearchCV(model_instance, gridSearch_params, n_jobs=-1, cv = cv_Stratified_KFold)    
grid_search_CV_Stratified_KFold_model.fit(x_train, y_train)        

best_model = grid_search_CV_Stratified_KFold_model.best_estimator_
y_pred = best_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)  
print('accuracy:', accuracy)
classification_rep = classification_report(y_test, y_pred)
print('classification_report:', classification_rep)
    

## Ejercicio 6 - Feature importance

Veamos qué importancia tiene cada una de las features del modelo entrenado en el ejercicio anterior

In [ ]:
feature_names = X.columns
feature_importance = best_model.feature_importances_
importance = pd.DataFrame({'feature_names': feature_names, 'feature_importance': feature_importance})
importance.sort_values(by = 'feature_importance', axis=0, ascending=False, inplace=True)
importance

## Ejercicio 7 - Extra

Calculen el área bajo la curva como medida de performance de alguno de los clasificadores entrenados

https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py


## Referencias 

Cross Validation
https://scikit-learn.org/stable/modules/cross_validation.html

StratifiedKFold
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold

Grid Search
https://scikit-learn.org/stable/modules/grid_search.html#grid-search

ROC
https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py

BaggingClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
